# Harris County Home Price Estimations

In [ ]:
import sqlite3

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from scipy.stats import shapiro
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

## Building and Real Account Data
This has the base features to include for the model. In this file I am only pulling the continuous data. So using the date built, improvement square feet, gross area, base area, land area, perimeter and size index to estimate the assessed_val.

In [ ]:
con = sqlite3.connect('HouseProtestValues.db')
sql_query = '''SELECT  br.acct,
                       br.bld_num,
                       br.date_erected,
                       br.im_sq_ft,
                       br.perimeter,
                       ra.land_val,
                       ra.bld_val,
                       ra.assessed_val,
                       ra.tot_appr_val
            FROM building_res as br
            LEFT JOIN real_acct as ra ON br.acct = ra.acct
            WHERE br.impr_tp = 1001 AND br.property_use_cd = 'A1' AND br.date_erected > 10;'''

base_df = pd.read_sql_query(sql_query, con)

## Fixtures Data
This has features such as, number of rooms such as bedrooms (RMB), full bath (RMF), half bath (RMH) and total rooms (RMT). This will be merged in a pandas dataframe based on the account number and building number. This data was storred in a table format with multiple accounts and building numbers for each feature, so I created a pivot table with the features as columns.

In [ ]:
# Story Height Index: STY
# Room: Bedroom: RMB
# Room: Full Bath: RMF
# Room: Half Bath: RMH
# Room: Total: RMT
fixtures_sql = """SELECT *
                  FROM "fixtures"
                  WHERE type IN ('STY', 'RMB','RMF','RMH','RMT')
                """
fixtures = pd.read_sql_query(fixtures_sql, con)

# Pivot table
fix_pt = fixtures.pivot_table(index=['acct', 'bld_num'], columns='type', values='units', aggfunc='sum')
fix_pt = fix_pt.reset_index()
fix_pt.fillna(0, inplace=True)

## Merge Base Data with Fixtures Pivot table

In [ ]:
data_df = pd.merge(base_df, fix_pt, on=['acct', 'bld_num'], how='left')
data_df.dropna(inplace=True)

## Reduce and sample data_df
There are over 1 million different residential houses that have data and that is too many to run the whole set on the model. I will use two techniques to reduce the data set, fist I will use a heuristic where account that has less than 50 square feet of improvements will be removed; these are mostly empty lots that have no livable domiciles. Next I will remove outliers that can skew the data by using the inner quartile range, and then I will have pandas pull a sample of the remaining. 
 * Remove rows where 'im_sq_ft' is less than 50. 
 * IQR: inner quartile range is a technique that is used to remove outliers. 

In [ ]:
# Heuristics
# Remove accounts with less than 50 square feet of improvement area
data_df = data_df[data_df['im_sq_ft'] > 50]

In [ ]:
# IQR
Q1 = data_df['assessed_val'].quantile(0.25)
Q3 = data_df['assessed_val'].quantile(0.75)
IQR = Q3 - Q1

lower_iqr = (Q1 - 1.5 * IQR)
upper_iqr = (Q3 + 1.5 * IQR)
print(f"Lower IQR: {lower_iqr} | Upper IQR: {upper_iqr}")

In [ ]:
# Filter data_df to values between Lower IQR and Upper IQR
reduced_df = data_df[data_df['assessed_val'] <= upper_iqr]

In [ ]:
sample_df = reduced_df.sample(n=5000, random_state=42)

x = sample_df[['date_erected', 'im_sq_ft', 'perimeter', 'RMB', 'RMF', 'RMH', 'RMT', 'STY']]
y = sample_df['assessed_val']


In [ ]:
print(f"All Data{sample_df.shape} | x {x.shape} | y {y.shape}")

In [ ]:
sample_df.describe()

In [ ]:
# Free up memory
base_df = None
fix_pt = None
fixtures = None

In [ ]:
# Histogram of assessed values
sns.histplot(data=sample_df, x="assessed_val")

In [ ]:
# Histogram with log transformation
sns.histplot(data=sample_df, x="assessed_val", log_scale=True)

In [ ]:
from scipy.stats import boxcox

transformed_data, lambda_value = boxcox(sample_df['assessed_val'])
sample_df['transformed_data'] = transformed_data
sns.histplot(data=sample_df, x="transformed_data")

## Test normality with Shapiro-Wilk Test
The Shapiro-Wilk test evaluates a data set and quantifies how likely it is that the data was sampled from a Gaussian distribution. This is believed to be a reliable test for naormality if the dataset is not too large, i.e. under 5,000.



In [ ]:
stat, p = shapiro(np.log(sample_df['assessed_val']))
print(f"H-null: the distribution is normal\nH-alternative: the distribution is not normal")
print(f'Statistic: {stat} | p: {p}')
alpha = 0.05  # General alpha for 95% confidence
if p > alpha:
    print(f'Sample looks normally distributed (fail to reject H-null).')
else:
    print('Sample does not look normally distributed (reject H-null in favor of H-alternative).')

In [ ]:
# Test if box cox transformation normalizes the data
stat, p = shapiro(sample_df['transformed_data'])
print(f"H-null: the distribution is normal\nH-alternative: the distribution is not normal")
print(f'Statistic: {stat} | p: {p}')
alpha = 0.05  # General alpha for 95% confidence
if p > alpha:
    print(f'Sample looks normally distributed (fail to reject H-null).')
else:
    print('Sample does not look normally distributed (reject H-null in favor of H-alternative).')

In [ ]:
corr_matrix = sample_df.corr()
plt.figure(figsize=(12, 10))
sns.heatmap(data=corr_matrix, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
sns.pairplot(data=sample_df, vars=['assessed_val', 'tot_appr_val', 'date_erected', 'im_sq_ft',
                                   'perimeter', 'RMB', 'RMF', 'RMH', 'RMT', 'STY'])

## Train, Test, Split!
The training and testing sets get split, but I will need to see some examples to see if there are indexes on the y's

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

## Feature Ranking

In [ ]:
rfc = RandomForestClassifier()

In [ ]:
rfc.fit(x_train, y_train)
importance = rfc.feature_importances_

In [ ]:
feature_importance = pd.Series(importance, index=x_train.columns)
print(feature_importance.sort_values(ascending=False))

In [ ]:
etc = ExtraTreesRegressor(random_state=42)

In [ ]:
etc.fit(x_train, y_train)
etc.feature_importances_

In [ ]:
cross_val_score(etc, x_train, y_train, cv=5, n_jobs=-1).mean()

In [ ]:
param_grid = {
    'n_estimators': [100, 750, 800, 1000, 5000],
    'min_samples_leaf': [0.25, 0.5, 1, 3, 4],
    'max_features': ["sqrt", "log2", 5, 7, 10, 15],
    'criterion': ['absolute_error', 'friedman_mse']
}

In [ ]:
# n_jobs will determine the amount of parallel process and how much memory will be used. -1 will use ALL cores, set to 50% of virtual cores to be safe and not use all the memory.
etc2 = GridSearchCV(etc, param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')

In [ ]:
etc2.fit(x_train, y_train)

In [ ]:
etc2.best_params_

In [ ]:
etc2.best_score_

In [ ]:
y_pred = etc2.predict(x_test)

In [ ]:
r2_score(y_test, y_pred)

In [ ]:
# Save model
import joblib

joblib.dump(etc2, 'etc.pkl')

# load
# joblib.load('etc.pkl')

# Residual Analysis

In [ ]:
actual = y_test.to_list()
predicted = etc2.predict(x_test)

act_pred_df = pd.DataFrame({'actual': actual, "predicted": predicted, })
act_pred_df['residuals'] = act_pred_df['actual'] - act_pred_df['predicted']
act_pred_df

In [ ]:
sns.scatterplot(data=act_pred_df, x='actual', y='predicted')

In [ ]:
sns.scatterplot(data=act_pred_df, x='actual', y='residuals')

## Gradient Boost
The assessed values are not normally distributed, but are skewed right. They will be log transformed and the predicted values will be expectationaled back to assess the residuals.

In [ ]:
sample_df['log(values)'] = np.log(sample_df['tot_appr_val'])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
gbr = GradientBoostingRegressor()

In [ ]:
param_grid = {
    'n_estimators': [100, 500, 1000],
    'learning_rate': [0.001, 0.0005, 0.002],
    'max_depth': [5, 10, 15],
    'min_samples_split': [3, 4, 6],
    'min_samples_leaf': [2, 3, 6],
    'criterion': ['friedman_mse'],
}

In [ ]:
gbr_cv = GridSearchCV(gbr, param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')

In [ ]:
gbr_cv.fit(x_train, y_train)
gbr_pred = gbr_cv.predict(x_test)

In [ ]:
gbr_mae = mean_absolute_error(y_test, gbr_pred)
gbr_mse = mean_squared_error(y_test, gbr_pred)
gbr_r2 = r2_score(y_test, gbr_pred)
print(f"MAE: {gbr_mae}\nMSE: {gbr_mse}\nR Squared: {gbr_r2}\n")

In [ ]:
gbr_cv.best_params_

In [ ]:
gbr_cv.best_score_

In [ ]:
joblib.dump(gbr_cv, 'gbr.pkl')

# load
# joblib.load('gbr.pkl')

In [ ]:
gbr_residual_df = pd.DataFrame({'actual': actual, "predicted": gbr_pred, })
gbr_residual_df['residuals'] = gbr_residual_df['actual'] - gbr_residual_df['predicted']

In [ ]:
sns.regplot(gbr_residual_df, x='actual', y='predicted')

In [ ]:
sns.regplot(gbr_residual_df, x='actual', y='residuals')

In [ ]:
import numpy as np

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

ax.scatter(gbr_residual_df['actual'], gbr_residual_df['predicted'], s=60, alpha=0.7, edgecolors='k')
b, a = np.polyfit(gbr_residual_df['actual'], gbr_residual_df['predicted'], 1)
ax.plot(gbr_residual_df['actual'], b * gbr_residual_df['actual'] + a)
ax.annotate(f"R-Squared = {gbr_r2}", (0, 1))
plt.show()